
# Convolutional Neural Network For Image Classification

- CNN이란?
- Convolutional Neural Network For Image Classification
  - Approach 1 :
  - Approach 2 :

## CNN(Convolutional Neural Network)
- CNN은
